In [ ]:
%%bash

pip install --no-cache-dir -qU scrapy

In [3]:
import os
import requests

base_dir = os.getcwd()
input_dir = os.path.join(base_dir, "Data/Input")
os.makedirs(input_dir, exist_ok=True)

url = "https://www.timeanddate.com/holidays"
html = requests.get(url).content

with open(os.path.join(input_dir, 'front.html'), 'wb') as file:
    file.write(html)

url = "https://www.timeanddate.com/holidays/afghanistan/2023?hol=1"
html = requests.get(url).content

country_dir = os.path.join(input_dir, "afghanistan")
os.makedirs(country_dir, exist_ok=True)
with open(os.path.join(country_dir, '2023.html'), 'wb') as file:
    file.write(html)

2023-09-17 04:02:44 [urllib3.connectionpool] DEBUG: Starting new HTTPS connection (1): www.timeanddate.com:443
2023-09-17 04:02:44 [urllib3.connectionpool] DEBUG: https://www.timeanddate.com:443 "GET /holidays HTTP/1.1" 301 244
2023-09-17 04:02:44 [urllib3.connectionpool] DEBUG: https://www.timeanddate.com:443 "GET /holidays/ HTTP/1.1" 200 None
2023-09-17 04:02:44 [urllib3.connectionpool] DEBUG: Starting new HTTPS connection (1): www.timeanddate.com:443
2023-09-17 04:02:44 [urllib3.connectionpool] DEBUG: https://www.timeanddate.com:443 "GET /holidays/afghanistan/2023?hol=1 HTTP/1.1" 200 None


In [ ]:
import os
import scrapy

input_dir = "./Data/Input"
with open(os.path.join(input_dir, 'front.html'), 'r') as file:
  html = file.read()

sel = scrapy.Selector(text=html)
blocks = sel.css("ul.category-list__list > li")
countries = blocks.xpath(".//text()").getall()
links = blocks.xpath("./a/@href").getall()
links

In [1]:
import os
import scrapy
from scrapy.crawler import CrawlerProcess
from scrapy.loader import ItemLoader
from itemloaders.processors import MapCompose, TakeFirst
from w3lib.html import remove_tags

base_dir = os.getcwd()
input_dir = "Data/Input"
output_dir = "Data/Output"

class HolidayItem(scrapy.Item):
    country = scrapy.Field(
        input_processor=MapCompose(remove_tags),
        output_processor=TakeFirst()
    )
    

class HolidaysSpider(scrapy.Spider):
    name = 'Holidays'

    def start_requests(self):
        urls = [
            f"file://{base_dir}/{input_dir}/front.html"
        ]
        for url in urls:
            yield scrapy.Request(url=url, callback=self.parse_front)

    def parse_front(self, response):
        for country in response.css('ul.category-list__list > li'):
            l = ItemLoader(item=HolidayItem(), selector=country)
            l.add_xpath("country", ".//text()")
            link = country.xpath("./a/@href").get()
            link = link + "/2023?hol=1"
            print(link)
            yield l.load_item()

    # def parse_pages(self, response):
    #     tables = response.css('table#holidays-table')
    #     dates = tables.xpath('.//th[@class="nw"]/text()').getall()
    #     print(dates)

process = CrawlerProcess(
    settings={
        "FEEDS":{f"{base_dir}/{output_dir}/holiday.jl":{"format":"jsonlines"}}
    }
)
process.crawl(HolidaysSpider)
process.start()

2023-09-17 03:41:08 [scrapy.utils.log] INFO: Scrapy 2.10.1 started (bot: scrapybot)
2023-09-17 03:41:08 [scrapy.utils.log] INFO: Versions: lxml 4.9.3.0, libxml2 2.10.3, cssselect 1.2.0, parsel 1.8.1, w3lib 2.1.2, Twisted 22.10.0, Python 3.8.10 (default, Mar 13 2023, 10:26:41) - [GCC 9.4.0], pyOpenSSL 23.2.0 (OpenSSL 3.1.2 1 Aug 2023), cryptography 41.0.3, Platform Linux-6.2.0-32-generic-x86_64-with-glibc2.29
2023-09-17 03:41:08 [scrapy.addons] INFO: Enabled addons:
[]
2023-09-17 03:41:08 [scrapy.crawler] INFO: Overridden settings:
{}
2023-09-17 03:41:08 [py.warnings] WARNING: /Work/venv/lib/python3.8/site-packages/scrapy/utils/request.py:248: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the default value. In other words, it is normal to get this warning if you have not defined a value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting. This is so for backward compatibility reasons, but it will change in a 

/holidays/afghanistan/
/holidays/albania/
/holidays/algeria/
/holidays/american-samoa/
/holidays/andorra/
/holidays/angola/
/holidays/anguilla/
/holidays/antigua-and-barbuda/
/holidays/argentina/
/holidays/armenia/
/holidays/aruba/
/holidays/australia/
/holidays/austria/
/holidays/azerbaijan/
/holidays/bahrain/
/holidays/bangladesh/
/holidays/barbados/
/holidays/belarus/
/holidays/belgium/
/holidays/belize/
/holidays/benin/
/holidays/bermuda/
/holidays/bhutan/
/holidays/bolivia/
/holidays/bosnia/
/holidays/botswana/
/holidays/brazil/
/holidays/british-virgin-islands/
/holidays/brunei/
/holidays/bulgaria/
/holidays/burkina-faso/
/holidays/burundi/
/holidays/cape-verde/
/holidays/cambodia/
/holidays/cameroon/
/holidays/canada/
/holidays/cayman-islands/
/holidays/central-african-republic/
/holidays/chad/
/holidays/chile/
/holidays/china/
/holidays/colombia/
/holidays/comores/
/holidays/republic-of-the-congo/
/holidays/dr-congo/
/holidays/cook-islands/
/holidays/costa-rica/
/holidays/ivory

2023-09-17 03:41:08 [scrapy.core.scraper] DEBUG: Scraped from <200 file:///Work/Web_scraping/CountryHoliday/Data/Input/front.html>
{'country': 'Georgia'}
2023-09-17 03:41:08 [scrapy.core.scraper] DEBUG: Scraped from <200 file:///Work/Web_scraping/CountryHoliday/Data/Input/front.html>
{'country': 'Germany'}
2023-09-17 03:41:08 [scrapy.core.scraper] DEBUG: Scraped from <200 file:///Work/Web_scraping/CountryHoliday/Data/Input/front.html>
{'country': 'Ghana'}
2023-09-17 03:41:08 [scrapy.core.scraper] DEBUG: Scraped from <200 file:///Work/Web_scraping/CountryHoliday/Data/Input/front.html>
{'country': 'Gibraltar'}
2023-09-17 03:41:08 [scrapy.core.scraper] DEBUG: Scraped from <200 file:///Work/Web_scraping/CountryHoliday/Data/Input/front.html>
{'country': 'Greece'}
2023-09-17 03:41:08 [scrapy.core.scraper] DEBUG: Scraped from <200 file:///Work/Web_scraping/CountryHoliday/Data/Input/front.html>
{'country': 'Greenland'}
2023-09-17 03:41:08 [scrapy.core.scraper] DEBUG: Scraped from <200 file:///

/holidays/georgia/
/holidays/germany/
/holidays/ghana/
/holidays/gibraltar/
/holidays/greece/
/holidays/greenland/
/holidays/grenada/
/holidays/guadeloupe/
/holidays/guam/
/holidays/guatemala/
/holidays/guernsey/
/holidays/guinea/
/holidays/guinea-bissau/
/holidays/guyana/
/holidays/haiti/
/holidays/honduras/
/holidays/hong-kong/
/holidays/hungary/
/holidays/iceland/
/holidays/india/
/holidays/indonesia/
/holidays/iran/
/holidays/iraq/
/holidays/ireland/
/holidays/isle-of-man/
/holidays/israel/
/holidays/italy/
/holidays/jamaica/
/holidays/japan/
/holidays/jersey/
/holidays/jordan/
/holidays/kazakhstan/
/holidays/kenya/
/holidays/kiribati/
/holidays/kosovo/
/holidays/kuwait/
/holidays/kyrgyzstan/
/holidays/laos/
/holidays/latvia/
/holidays/lebanon/
/holidays/lesotho/
/holidays/liberia/
/holidays/libya/
/holidays/liechtenstein/
/holidays/lithuania/
/holidays/luxembourg/
/holidays/macau/
/holidays/madagascar/
/holidays/malawi/
/holidays/malaysia/
/holidays/maldives/
/holidays/mali/
/holi

2023-09-17 03:41:08 [scrapy.core.scraper] DEBUG: Scraped from <200 file:///Work/Web_scraping/CountryHoliday/Data/Input/front.html>
{'country': 'Norway'}
2023-09-17 03:41:08 [scrapy.core.scraper] DEBUG: Scraped from <200 file:///Work/Web_scraping/CountryHoliday/Data/Input/front.html>
{'country': 'Oman'}
2023-09-17 03:41:08 [scrapy.core.scraper] DEBUG: Scraped from <200 file:///Work/Web_scraping/CountryHoliday/Data/Input/front.html>
{'country': 'Pakistan'}
2023-09-17 03:41:08 [scrapy.core.scraper] DEBUG: Scraped from <200 file:///Work/Web_scraping/CountryHoliday/Data/Input/front.html>
{'country': 'Palau'}
2023-09-17 03:41:08 [scrapy.core.scraper] DEBUG: Scraped from <200 file:///Work/Web_scraping/CountryHoliday/Data/Input/front.html>
{'country': 'Panama'}
2023-09-17 03:41:08 [scrapy.core.scraper] DEBUG: Scraped from <200 file:///Work/Web_scraping/CountryHoliday/Data/Input/front.html>
{'country': 'Papua New Guinea'}
2023-09-17 03:41:08 [scrapy.core.scraper] DEBUG: Scraped from <200 file:/

/holidays/norway/
/holidays/oman/
/holidays/pakistan/
/holidays/palau/
/holidays/panama/
/holidays/papua-new-guinea/
/holidays/paraguay/
/holidays/peru/
/holidays/philippines/
/holidays/poland/
/holidays/portugal/
/holidays/puerto-rico/
/holidays/qatar/
/holidays/reunion/
/holidays/romania/
/holidays/russia/
/holidays/rwanda/
/holidays/saint-helena/
/holidays/saint-kitts-and-nevis/
/holidays/saint-lucia/
/holidays/saint-martin/
/holidays/saint-pierre-and-miquelon/
/holidays/saint-vincent-and-the-grenadines/
/holidays/samoa/
/holidays/san-marino/
/holidays/sao-tome-and-principe/
/holidays/saudi-arabia/
/holidays/senegal/
/holidays/serbia/
/holidays/seychelles/
/holidays/sierra-leone/
/holidays/singapore/
/holidays/sint-maarten/
/holidays/slovakia/
/holidays/slovenia/
/holidays/solomon-islands/
/holidays/somalia/
/holidays/south-africa/
/holidays/south-korea/
/holidays/south-sudan/
/holidays/spain/
/holidays/sri-lanka/
/holidays/saint-barthelemy/
/holidays/sudan/
/holidays/suriname/
/hol

2023-09-17 03:41:09 [scrapy.core.scraper] DEBUG: Scraped from <200 file:///Work/Web_scraping/CountryHoliday/Data/Input/front.html>
{'country': 'UN International Years'}
2023-09-17 03:41:09 [scrapy.core.scraper] DEBUG: Scraped from <200 file:///Work/Web_scraping/CountryHoliday/Data/Input/front.html>
{'country': 'Fun Holidays'}
2023-09-17 03:41:09 [scrapy.core.engine] INFO: Closing spider (finished)
2023-09-17 03:41:09 [scrapy.extensions.feedexport] INFO: Stored jsonlines feed (237 items) in: /Work/Web_scraping/CountryHoliday/Data/Output/holiday.jl
2023-09-17 03:41:09 [scrapy.statscollectors] INFO: Dumping Scrapy stats:
{'downloader/request_bytes': 261,
 'downloader/request_count': 1,
 'downloader/request_method_count/GET': 1,
 'downloader/response_bytes': 72123,
 'downloader/response_count': 1,
 'downloader/response_status_count/200': 1,
 'elapsed_time_seconds': 0.796519,
 'feedexport/success_count/FileFeedStorage': 1,
 'finish_reason': 'finished',
 'finish_time': datetime.datetime(2023

/year/
/holidays/fun/


In [ ]:
import scrapy
import requests

url = 'https://www.timeanddate.com/holidays/afghanistan/2023?hol=1'
html = requests.get(url).content

sel = scrapy.Selector(text=html)
tables = sel.css('table#holidays-table')
tables
# dates = sel.css('th.nw::text').extract()
# holidays = sel.css('td > a::text').extract()
